In [ ]:
# # Main repo here for pytorch
# # https://github.com/timesler/facenet-pytorch

# from __future__ import print_function, division

# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.optim import lr_scheduler
# import numpy as np
# import torchvision
# from torchvision import datasets, models, transforms
# import matplotlib.pyplot as plt
# import time
# import os
# import cv2
# import copy
# from facenet_pytorch import InceptionResnetV1

# plt.ion()   # interactive mode

# # Data augmentation and normalization for training
# # Just normalization for validation
# data_transforms = {
#     'train': transforms.Compose([
#         transforms.RandomResizedCrop(299),
#         #transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
#     'val': transforms.Compose([
#         transforms.Resize(299),
#         transforms.CenterCrop(299),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
# }

# data_dir = './dataset/'
# image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
#                                           data_transforms[x])
#                   for x in ['train', 'val']}
# dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
#                                              shuffle=True, num_workers=4)
#               for x in ['train', 'val']}
# dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

# print(dataset_sizes)

# class_names = image_datasets['train'].classes

# print(dataloaders['train'])

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("Using " + str(device))

# # sigmoid_fun = torch.nn.Sigmoid()
# # Remove the manual sigmoid function
# # sigmoid_fun = torch.nn.Sigmoid()

# # In the train_model function, change this line
# # loss = criterion(outputs[:,0], labels.float())

# # to
# loss = criterion(outputs, labels.float())


# def imshow(inp, title=None):
#     """Imshow for Tensor."""
#     inp  = inp.numpy().transpose((1, 2, 0))
#     mean = np.array([0.485, 0.456, 0.406])
#     std  = np.array([0.229, 0.224, 0.225])
#     inp  = std * inp + mean
#     inp  = np.clip(inp, 0, 1)
#     plt.imshow(inp)
#     if title is not None:
#         plt.title(title)
#     plt.pause(5)  # pause a bit so that plots are updated


# # Get a batch of training data
# inputs, classes = next(iter(dataloaders['train']))

# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs)

# def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
#     since = time.time()

#     best_model_wts = copy.deepcopy(model.state_dict())
#     best_acc = 0.0
#     for epoch in range(num_epochs):
#         print('Epoch {}/{}'.format(epoch, num_epochs - 1))
#         print('-' * 10)

#         # Each epoch has a training and validation phase
#         for phase in ['train', 'val']:
#             if phase == 'train':
#                 model.train()  # Set model to training mode
#             else:
#                 model.eval()   # Set model to evaluate mode

#             running_loss = 0.0
#             running_corrects = 0

#             # Iterate over data.
#             for inputs, labels in dataloaders[phase]:
#                 inputs = inputs.to(device)
#                 labels = labels.to(device)

#                 # zero the parameter gradients
#                 optimizer.zero_grad()

#                 # forward
#                 # track history if only in train
#                 with torch.set_grad_enabled(phase == 'train'):
#                     outputs = model(inputs)
#                     preds = sigmoid_fun(outputs[:,0]) > 0.5
#                     preds = preds.float()  
#                     loss = criterion(outputs[:,0], labels.float())
#                     if phase == 'train':
#                         loss.backward()
#                         optimizer.step()

#                 # statistics
#                 running_loss += loss.item() * inputs.size(0)
#                 running_corrects += torch.sum(preds == labels.data)
#             if phase == 'train':
#                 scheduler.step()

#             epoch_loss = running_loss / dataset_sizes[phase]
#             epoch_acc = running_corrects.double() / dataset_sizes[phase]

#             print('{} Loss: {:.4f} Acc: {:.4f}'.format(
#                 phase, epoch_loss, epoch_acc))

#             # deep copy the model
#             if phase == 'val' and epoch_acc > best_acc:
#                 best_acc = epoch_acc
#                 best_model_wts = copy.deepcopy(model.state_dict())

#         print()

#     time_elapsed = time.time() - since
#     print('Training complete in {:.0f}m {:.0f}s'.format(
#         time_elapsed // 60, time_elapsed % 60))
#     print('Best val Acc: {:4f}'.format(best_acc))

#     # load best model weights
#     model.load_state_dict(best_model_wts)
#     return model

# def visualize_model(model, num_images=6):
#     was_training = model.training
#     model.eval()
#     images_so_far = 0
#     fig = plt.figure()

#     with torch.no_grad():
#         for i, (inputs, labels) in enumerate(dataloaders['val']):
#             inputs = inputs.to(device)
#             labels = labels.to(device)
            
#             outputs = model(inputs)
#             preds = sigmoid_fun(outputs[:,0]) > 0.5
#             preds = torch.tensor(preds, dtype=torch.uint8)

#             for j in range(inputs.size()[0]):
#                 images_so_far += 1
#                 ax = plt.subplot(num_images//2, 2, images_so_far)
#                 ax.axis('off')
#                 ax.set_title('predicted: {}'.format(class_names[preds[j]]))
#                 imshow(inputs.cpu().data[j])

#                 if images_so_far == num_images:
#                     model.train(mode=was_training)
#                     return
#         model.train(mode=was_training)

# model_ft = InceptionResnetV1(pretrained='vggface2', device=device, classify= True, num_classes=1)

# print(model_ft.logits)

# model_ft = model_ft.to(device)

# criterion = nn.BCEWithLogitsLoss()

# # Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# # Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
#                        num_epochs=20)

# model_path = "trained_model.pt"
# print("Saving model "+model_path);
# torch.save(model_ft.state_dict(), model_path)


In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from facenet_pytorch import InceptionResnetV1

plt.ion()   # interactive mode

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = './dataset/'  # Change this to your dataset directory
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                              shuffle=True, num_workers=4)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using " + str(device))

model_ft = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes=len(class_names))
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                # Track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Train the model
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)

# Save the trained model
model_path = "trained_model.pt"
print("Saving model "+model_path)
torch.save(model_ft.state_dict(), model_path)

# Now, you can use this trained model for inference on new data


Using cuda
Epoch 0/19
----------
train Loss: 2.3865 Acc: 0.1765
val Loss: 2.1620 Acc: 0.2303

Epoch 1/19
----------
train Loss: 2.2876 Acc: 0.1913
val Loss: 2.2148 Acc: 0.2039

Epoch 2/19
----------
train Loss: 2.2531 Acc: 0.2028
val Loss: 2.0856 Acc: 0.2697

Epoch 3/19
----------
train Loss: 2.1541 Acc: 0.2307
val Loss: 2.1698 Acc: 0.2895

Epoch 4/19
----------
train Loss: 2.1164 Acc: 0.2250
val Loss: 1.8201 Acc: 0.3750

Epoch 5/19
----------
train Loss: 2.1171 Acc: 0.2496
val Loss: 1.8146 Acc: 0.3684

Epoch 6/19
----------
train Loss: 2.0084 Acc: 0.2791
val Loss: 1.8905 Acc: 0.3618

Epoch 7/19
----------
train Loss: 1.8251 Acc: 0.3456
val Loss: 1.4073 Acc: 0.4868

Epoch 8/19
----------
train Loss: 1.6877 Acc: 0.3768
val Loss: 1.2984 Acc: 0.5263

Epoch 9/19
----------
train Loss: 1.6565 Acc: 0.4154
val Loss: 1.2925 Acc: 0.5461

Epoch 10/19
----------
train Loss: 1.6244 Acc: 0.4031
val Loss: 1.2003 Acc: 0.5724

Epoch 11/19
----------
train Loss: 1.6122 Acc: 0.4228
val Loss: 1.1394 Acc:

In [25]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# Load the model
model = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes=8)
model.load_state_dict(torch.load("trained_model.pt"))
model.eval()

# Define transformations for the input image
transform = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load and preprocess the input image
image_path = "012_Ajay20Devgan.jpg"
image = Image.open(image_path)
input_tensor = transform(image)
input_batch = input_tensor.unsqueeze(0)  # Add batch dimension

# Move input tensor to the appropriate device (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
input_batch = input_batch.to(device)  # Corrected variable name

# Perform inference
with torch.no_grad():
    output = model(input_batch)

# Process the output (if needed)
# For example, if your model returns probabilities, you might want to apply softmax and get class labels

# Output the result
print(output)


tensor([[ 3.0458,  1.2683, -0.7745, -1.3613,  0.3306,  2.3833, -1.8055, -3.3548]],
       device='cuda:0')


In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# Load the model
model = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes=1)
model.load_state_dict(torch.load("trained_model.pt"))
model.eval()

# Define transformations for the input image
transform = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load and preprocess the input image
image_path = "04_01_2023-hrithik_roshan_23283097.jpg"
image = Image.open(image_path)
input_tensor = transform(image)
input_batch = input_tensor.unsqueeze(0)  # Add batch dimension

# Move input tensor to the appropriate device (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
input_batch = input_batch.to(device)

# Perform inference
with torch.no_grad():
    output = model(input_batch)

# Apply sigmoid function to convert logits to probabilities
probabilities = torch.sigmoid(output)

# Assign label based on probability
if probabilities.item() >= 0.5:
    label = "Class 1"  # Example label for positive class
else:
    label = "Class 0"  # Example label for negative class

# Output the result with the associated label
print("Predicted label:", label)


In [28]:
import torch
from torchvision.transforms import transforms
from PIL import Image
from facenet_pytorch import InceptionResnetV1

# Load the pre-trained model
model = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes=8)

# Load the trained weights
model.load_state_dict(torch.load("trained_model.pt"))

# Set the model to evaluation mode
model.eval()

# Define transformations for the input image
transform = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load and preprocess the input image
image_path = "mb.jpeg"
image = Image.open(image_path)
input_tensor = transform(image)
input_batch = input_tensor.unsqueeze(0)  # Add batch dimension

# Move input tensor to the appropriate device (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
input_batch = input_batch.to(device)

# Perform inference
with torch.no_grad():
    output = model(input_batch)

# Define class names
class_names = ['Ajay_Devgan', 'Hrithik_Roshan', 'Leonardo_DiCaprio','Mahesh_Babu','Shah_Rukh_Khan','Shahid_Kapoor','Tom_Cruise','Tom_Hanks']  # Replace with your actual class names

# Get the predicted class index
predicted_class_index = torch.argmax(output, dim=1).item()

# Map the predicted class index to the actual class label
predicted_label = class_names[predicted_class_index]

# Print the output with the associated class name
print("Predicted class:", predicted_label)


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x0000020F483620C0>
Traceback (most recent call last):
  File "c:\Users\shree\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "c:\Users\shree\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py", line 1437, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
                                   ^^^^^^^^^^^^^^^^^^^^
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


Predicted class: Hrithik_Roshan
